In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 9809 (delta 213), reused 301 (delta 206), pack-reused 9487
Receiving objects: 100% (9809/9809), 9.57 MiB | 22.08 MiB/s, done.
Resolving deltas: 100% (7221/7221), done.
/content/deep-person-reid
     |████████████████████████████████| 5.8 MB 5.4 MB/s 
     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 185 kB 43.5 MB/s 
     |████████████████████████████████| 42 kB 935 kB/s 
     |████████████████████████████████| 69 kB 8.1 MB/s 
     |████████████████████████████████| 42 kB 829 kB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.8.2
    Uninstalling importlib-metadata-4.8.2:
      Successfully uninstalled importlib-metadata-4.8.2
/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython ev

In [3]:
import torchreid

In [4]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3865 KB/s, 38 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [5]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [7]:
engine.run(
    save_dir='log/resnet50',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/60][10/404]	time 0.460 (1.029)	data 0.001 (0.557)	eta 6:55:36	loss 6.9783 (6.8005)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/60][20/404]	time 0.457 (0.744)	data 0.001 (0.279)	eta 5:00:12	loss 6.7539 (6.8758)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/60][30/404]	time 0.459 (0.649)	data 0.000 (0.186)	eta 4:21:54	loss 6.7056 (6.8091)	acc 0.0000 (0.4167)	lr 0.000300
epoch: [1/60][40/404]	time 0.461 (0.602)	data 0.001 (0.140)	eta 4:02:43	loss 6.4921 (6.7678)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/60][50/404]	time 0.458 (0.573)	data 0.002 (0.112)	eta 3:51:11	loss 6.5320 (6.7484)	acc 0.0000 (0.5000)	lr 0.000300
epoch: [1/60][60/404]	time 0.458 (0.555)	data 0.000 (0.094)	eta 3:43:30	loss 6.5386 (6.7204)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/60][70/404]	time 0.458 (0.541)	data 0.000 (0.080)	eta 3:38:01	loss 6.7234 (6.7092)	acc 0.0000 (0.4018)	lr 0.000300
epoch: [1/60][80/404]	time 0.459 (0.531)	data 0.001 (0.070)	eta 3:33:52	loss 6.5081 (6.6952)	acc 0.0000 (0.3906)	lr 0.000300
